In [6]:
import numpy as np
import matplotlib.pyplot as plt

import sympy as sp
from scipy.optimize import linprog, minimize_scalar
from sympy import symbols, sympify, diff, Eq, solve
import re

In [11]:
class FeasibleDirections():
    def __init__(self, x_array, x0_input, subs, limitations, error, target):
        self.x_array = x_array
        self.x0_input = x0_input
        self.subs = subs
        self.limitations = limitations
        self.error = error
        self.target = target
        self.prev = None

    def print_vars(self):
        print('self.x_array', self.x_array)
        print('self.x0_input', self.x0_input)
        print('self.subs', self.subs)
        print('self.limitations', self.limitations)
        print('self.error', self.error)
        print('self.target', self.target)

    
    # self.x_array [x1, x2] <class 'list'> <class 'sympy.core.symbol.Symbol'> # переменные 
    # self.x0_input (-1, 1) <class 'tuple'> # стартовая точка
    # nabla_f_res # градиент по переменным
    # self.subs [(x1, 3), (x2, 1)] # пары переменных с их координатами. по мере алгоритма меняется
    # self.limitations [x1 + x2 - 4, -x1 - 2*x2 + 2, -x1, -x2] <class 'list'> <class 'sympy.core.add.Add'> #  
    # limitations_res = [limit.subs(self.subs) for limit in self.limitations] # подстановка градиента в ограничения

    # self.error # Значение ошибки, епсилон
    # self.target -x1**2 + x1*x2 + 4*x1 - 2*x2**2 + 6*x2 <class 'sympy.core.add.Add'> # Целевая функция
    def run_iteration(self, k):
        # try:
        # Step 1
        nabla_f_res = [diff(self.target, x).subs(self.subs) for x in self.x_array]
        #print(f'\n!!!!!!!!!!!!!!!!\nnabla_f_res: {nabla_f_res}')
        limitations_res = [limit.subs(self.subs) for limit in self.limitations]
        #print('self.subs', self.subs, type(self.subs))
        #print('limitations', self.limitations, type(self.limitations), type(self.limitations[0]), '\nlimitations_res', limitations_res, type(limitations_res), '\n!!!!!!!!!!!!!!!!!\n')
        print(f"Gradient vector: ∇f(x{k})={nabla_f_res}")

        # Step 2
        #print('self.error', self.error)
        limitations_indexes = []
        for i, limit_res in enumerate(limitations_res):
            print(f'\tg{i}(x{k})={limit_res}')
            if abs(limit_res) <= self.error:
                limitations_indexes.append(i)
        print(f"Limitation indexes: {limitations_indexes}")

        # Step 3
        #print('self.target', self.target, type(self.target))
        if len(limitations_indexes) == 0:
            # Step 5
            s_k = nabla_f_res
            omega_k = sum(k ** 2 for k in nabla_f_res) ** (1 / 2)
        else:
            # Step 4
            omega = symbols('omg')
            s = symbols(' '.join([f's{i + 1}' for i in range(len(self.x_array))]))

            # target for inner
            c = [float(-omega.coeff(omega))] + [0.0] * len(s)  # -omega because linprog minimizes

            nabla_f = [diff(self.target, x) for x in self.x_array]
            extra_limit_expr = -sum(nabla_i * s_i for nabla_i, s_i in zip(nabla_f, s)) + omega
            limitations_inner = [extra_limit_expr]
            for index in limitations_indexes:
                nabla_g = [diff(self.limitations[index], x) for x in self.x_array]
                limitations_inner.append(
                    self.limitations[index] + sum(nabla_i * s_i for nabla_i, s_i in zip(nabla_g, s)) + omega)
            limitations_inner = list(map(lambda x: x.subs(self.subs), limitations_inner))

            # left part of limitations
            A = [
                [float(expr.coeff(x)) for x in [omega] + list(s)]
                for expr in limitations_inner
            ]
            b = [0.0] * len(limitations_inner)

            # Решение задачи линейного программирования
            result = linprog(c, A_ub=A, b_ub=b,
                            bounds=[(float('-inf'), None)] + [(-1, 1) for _ in range(len(c) - 1)])
            # print(f'Left part inner: {A}')
            # print(f'Right part inner: {b}')

            omega_k = result.x[0]
            s_k = result.x[1:]

        print(f"\tσk: {omega_k}")
        print(f"\tSk: {s_k}")
        # Step 6
        if abs(omega_k) < self.error:
            print('Found result!')
            return True, self.subs
        self.prev = self.target.subs(self.subs)

        # Step 7
        # Step 7.1.
        beta = symbols('beta')
        betas_l = []
        for limit in self.limitations:
            equation = Eq(limit.subs([(sub[0], sub[1] + beta * s_k[i]) for i, sub in enumerate(self.subs)]), 0)
            solution = list(map(float, solve(equation, beta)))
            betas_l.extend(filter(lambda x: x > 0, solution))
        beta_star = min(betas_l)
        print(f"\tBeta*: {beta_star}")

        # Step 7.2

        def objective_function(beta):
            return -float(self.target.subs([(sub[0], sub[1] + beta * s_k[i]) for i, sub in enumerate(self.subs)]))

        # Find the maximum value and the corresponding x
        result = minimize_scalar(objective_function, bounds=(0.0, beta_star), method='bounded')

        self.subs = [(sub[0], sub[1] + result.x * s_k[i]) for i, sub in enumerate(self.subs)]
        print(f'Current: {self.subs}')

        return False, []
        # except BaseException as e:
        #     print(f"Unexpected error: {e}")
        #     return True, None

    def run(self):
        self.k = 0
        while True:
            print(f"{'-' * 30}Iteration {self.k + 1}{'-' * 30}")
            feedback, result = self.run_iteration(self.k)
            if feedback:
                break
            if self.k > 1000:
                print("!!!!!!!!!!!!! Simulation execution taken too many iterations !!!!!!!!!!!!!")
                return
            self.k += 1
        print(f"{'-' * 26}Simulation ended!{'-' * 26}")
        if result is None:
            print(f"Simulation stopped unexpectedly")
        else:
            print(f"x*={result}\nf(x*)={-self.target.subs(result)}\n")

In [12]:
# self.x_array [x1, x2] <class 'list'> <class 'sympy.core.symbol.Symbol'> # переменные 
# self.x0_input (-1, 1) <class 'tuple'> # стартовая точка
# nabla_f_res # градиент по переменным
# self.subs [(x1, 3), (x2, 1)] # пары переменных с их координатами. по мере алгоритма меняется
# limitations [x1 + x2 - 4, -x1 - 2*x2 + 2, -x1, -x2] <class 'list'> <class 'sympy.core.add.Add'> #  
# limitations_res = [limit.subs(self.subs) for limit in self.limitations] # подстановка градиента в ограничения

# self.error # Значение ошибки, епсилон
# self.target -x1**2 + x1*x2 + 4*x1 - 2*x2**2 + 6*x2 <class 'sympy.core.add.Add'> # Целевая функция

In [13]:
target = sympify('x1**2 - x1*x2 - 4*x1 + 2*x2**2 - 6*x2')

x_array = symbols(['x1', 'x2'])
x0_input = (3, 1)
subs = [(var, num) for var, num in zip(x_array, x0_input)]

limitations_str = [
    'x1 + x2 - 4',
    '-x1 - 2*x2 + 2',
    '-x1',
    '-x2',
]
limitations = [sympify(expr) for expr in limitations_str]

error = 0.1

print('target:', target)
print('limitations:', limitations)
print('vars:', x_array)
print('start:', x0_input)
print('error:', error)
print('\nsubs:', subs)

target: x1**2 - x1*x2 - 4*x1 + 2*x2**2 - 6*x2
limitations: [x1 + x2 - 4, -x1 - 2*x2 + 2, -x1, -x2]
vars: [x1, x2]
start: (3, 1)
error: 0.1

subs: [(x1, 3), (x2, 1)]


In [14]:
solution = FeasibleDirections(x_array, x0_input, subs, limitations, error, target)
solution.print_vars()

self.x_array [x1, x2]
self.x0_input (3, 1)
self.subs [(x1, 3), (x2, 1)]
self.limitations [x1 + x2 - 4, -x1 - 2*x2 + 2, -x1, -x2]
self.error 0.1
self.target x1**2 - x1*x2 - 4*x1 + 2*x2**2 - 6*x2


In [15]:
solution = FeasibleDirections(x_array, x0_input, subs, limitations, error, target)
solution.run()

------------------------------Iteration 1------------------------------
Gradient vector: ∇f(x0)=[1, -5]
	g0(x0)=0
	g1(x0)=-3
	g2(x0)=-3
	g3(x0)=-1
Limitation indexes: [0]
	σk: 2.0
	Sk: [-1. -1.]
	Beta*: 1.0
Current: [(x1, 2.0000059608609866), (x2, 5.96086098658688e-06)]
------------------------------Iteration 2------------------------------
Gradient vector: ∇f(x1)=[5.96086098658688e-6, -7.99998211741704]
	g0(x1)=-1.99998807827803
	g1(x1)=-0.0000178825829597606
	g2(x1)=-2.00000596086099
	g3(x1)=-0.00000596086098658688
Limitation indexes: [1, 3]
	σk: 6.62319203396109e-07
	Sk: [1.00000000e+00 6.62319203e-07]
Found result!
--------------------------Simulation ended!--------------------------
x*=[(x1, 2.0000059608609866), (x2, 5.96086098658688e-06)]
f(x*)=4.00004768681683

